#![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png) + ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)
#Practica sobre como usar Machine learning no supervisado (clustering) para realizar una segmentacion comportamental

Esta practica simula un ejercicio completo sobre como generar grupos (clusters) de clientes usando un dataset de usuarios y ratings liberados por una empresa de streaming multimedia bajo demanda por Internet.

**Este notebook cubre:**
* *Parte 1:* ETL de los ficheros (3 puntos)
* *Parte 2:* Ingenieria de caracteristicas (3 puntos)
* *Parte 3:* Clustering (2 puntos)
* *Parte 4:* Validacion de los resultados (2 punto)
* *Parte 5:* Normalizacion de los datos --Opcional--

**IMPORTANTE:** Usar un cluster de spark 2.0 para realizar esta PEC

## Parte 1: ETL de los ficheros

** Resumen **

Los archivos que usaremos contienen 1.000.209 calificaciones anonimas de aproximadamente 3.900 peliculas Realizado por 6.040 usuarios realizadas durante el ano 2000.

** Fichero de calificaciones **

Todas las calificaciones estan contenidas en el archivo "ratings.dat" y estan en el siguiente formato:

*UserID::MovieID::Rating::Timestamp*

- UserIDs: cuyo rango se encuentra entre 1 y 6040
- MovieIDs: cuyo rango se encuentra entre 1 y 3952
- Rating: las calificaciones se realizan en una escala de 5 estrellas (sin decimales)
- Timestamp: la marca de tiempo se representa en segundos

Nota: Cada usuario tiene al menos 20 calificaciones

** Fichero de usuarios **

La informacion sobre los usuarios esta contenida en el fichero "users.dat" y estan en el siguiente formato:

*UserID::Gender::Age::Occupation::Zip-code*

Toda la informacion demografica es proporcionada voluntariamente por los usuarios y no se comprueba su exactitud. Solo los usuarios que hayan proporcionado datos demograficos se incluyen en este conjunto de datos.

- Gender: El genero se denota por "M" para hombres y "F" para mujeres
- Edad: Esta distibuida en los siguientes rangos:

	*  1:  "- 18"
	* 18:  "18-24"
	* 25:  "25-34"
	* 35:  "35-44"
	* 45:  "45-49"
	* 50:  "50-55"
	* 56:  "56 - "

- Profesion: Se elige de las siguientes opciones:

	*  0:  "other" or not specified
	*  1:  "academic/educator"
	*  2:  "artist"
	*  3:  "clerical/admin"
	*  4:  "college/grad student"
	*  5:  "customer service"
	*  6:  "doctor/health care"
	*  7:  "executive/managerial"
	*  8:  "farmer"
	*  9:  "homemaker"
	* 10:  "K-12 student"
	* 11:  "lawyer"
	* 12:  "programmer"
	* 13:  "retired"
	* 14:  "sales/marketing"
	* 15:  "scientist"
	* 16:  "self-employed"
	* 17:  "technician/engineer"
	* 18:  "tradesman/craftsman"
	* 19:  "unemployed"
	* 20:  "writer"
    
** Fichero de peliculas **

La informacion se ecuentra en el archivo "movies.dat" y esta en el siguiente
formato:

*MovieID::Title::Genres*

- Title: Los titulos son identicos a los titulos proporcionados por la IMDB (incluyendo ano de lanzamiento)
- Genres: Los generos estan separados por *pipes* y se seleccionan de la siguiente lista:
    * Action
	* Adventure
	* Animation
	* Children's
	* Comedy
	* Crime
	* Documentary
	* Drama
	* Fantasy
	* Film-Noir
	* Horror
	* Musical
	* Mystery
	* Romance
	* Sci-Fi
	* Thriller
	* War
	* Western
    
- Algunos MovieIDs no corresponden a una pelicula debido a un duplicado accidental a su entradas y/o a entradas de prueba
- Las peliculas se introducen principalmente a mano, por lo que pueden existir errores e incoherencias

**Ejercicio 1(a):** Empezaremos por visualizar una muestra de los datos. Para esto usaremos las funciones pre-definidas en los notebooks de Databricks para explorar su sistema de archivos.

Usar `display(dbutils.fs.ls("/databricks-datasets/cs110x/ml-1m/data-001")` para listar los ficheros del directorio%md descripcion de los datos que se van a usar y decir que los vamos a cargar

In [4]:
#TODO: use display to list all the files of the directory containing the data
<FILL IN>


Luego usaremos el comando `print dbutils.fs.head` para visualizar el contenido de los ficheros "movies.dat", "user.dat" y "ratings.dat"

In [6]:
#TODO: use dbutils.fs.head to inspect the file "user.dat"
<FILL IN>

In [7]:
#TODO: use dbutils.fs.head to inspect the file "ratings.dat"
<FILL IN>

In [8]:
#TODO: use dbutils.fs.head to inspect the file "movies.dat"
<FILL IN>


## Ejercicio 1(b):

Carga los diferentes ficheros (usuarios, peliculas y calificaciones) en tres rdds para poder procesarlos posteriormente.

In [10]:
# TODO: Load the users data and print the first five lines.
rawUsersTextRdd = <FILL IN>
print <FILL IN>

In [11]:
# TODO: Load the ratings data and print the first five lines.
rawRatingsTextRdd = <FILL IN>
print <FILL IN>

In [12]:
# TODO: Load the movies data and print the first five lines.
rawMoviesTextRdd = <FILL IN>
print <FILL IN>

### Ejercicio 1(c): 
Como has observado inspecionando los ficheros, el fichero de usuarios contiene algunas lineas con mas de un zip-code. Para simplificar esta PEC, eliminaremos estas lineas del fichero. Para eso crearemos una funcion que elimine todas las lineas que contenga algun caracter que no sea '0123456789:MF'.

In [14]:
#TODO: Complete the function invalidLine
def invalidLine(line):
    """Verifies if a line is valid to be converted to a dataframe.
    Args:
        line (str): A string.

    Returns:
        boolean: True if valid, False otherwise.
    """
  <FILL IN>

In [15]:
# Load in the testing code and check to see if your answer is correct
# If incorrect it will report back '1 test failed' for each failed test
# Make sure to rerun any cell you change before trying the test again
from databricks_test_helper import Test
# TEST invalidLine (1b)
Test.assertEquals(invalidLine('161::M::45::16::98107-2117'), False, 'incorrect result: invalidLine does not remove the incorrect lines')

### Exercicio 1(d)

Como en la PEC anterior, crea un esquema a medida para cada uno de los ficheros. Puedes usar un codigo muy similar al de la PEC2 para esto.

In [17]:
#TODO: Fill in the user schema.
from pyspark.sql.types import *

# Custom Schema for users
userSchema = StructType([ \
    <FILL IN>
                          ])

In [18]:
# Custom Schema for ratings
ratingsSchema = StructType([ \
    <FILL IN>
                           ])

In [19]:
# Custom Schema for movies
moviesSchema = StructType([ \
    <FILL IN>
                          ])

## Ejercicio 1(d)

Elimina las lineas no validas contenidas en rawUsersTextRdd.

In [21]:
#TODO: filter invalid lines
filteredUsersTextRdd = <FILL IN>

In [22]:
# TEST filter invalidLines (1d)
counter = filteredUsersTextRdd.count()
Test.assertEquals(counter, 5974, 'incorrect result: lines are incorrectly filtered')

## Ejercicio 1(d)

Para poder usar facilmente diferentes algoritmos de clustering es conveniente que todos los atributos sean numericos, para esto convertiremos el atributo gender de los usuarios en 1 si es hombre ('M') o 0 si es mujer ('F'), cambia en el esquema que has creado anteriormente el atributo gener a `IntegerType()` si lo tenias como char o string.

Luego transforma el RDD en un DataFrame usando la funcion [toDF()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.SQLContext) indicandole que schema debe usar como un parametro de la funcion (`schema= customSchema`).

Para realizar esta transformacion a DataFrame, primero hay que convertir cada linea en una lista de enteros, ya que asi se indica en nuestro esquema. Para esto usaremos las funciones de Python [split()](https://docs.python.org/2/library/stdtypes.html#str.split) y [int()](https://docs.python.org/2/library/functions.html#int). Puedes combinar estas dos transformaciones junto con la conversion del atributo gender en la misma funcion lambda o hacer una funcion con nombre. Luego usa una funcion map para aplicar los cambios a todas las lineas del RDD de usuarios.

Finalmente, comprueba que los datos mostrados por el comando `display(usersDF)` son correctos.

In [24]:
#TODO: transform the Gender type and create a DataFrame
integerGendersUsersTextRdd = filteredUsersTextRdd.<FILL IN>
usersDF = <FILL IN>

display(usersDF)

## Ejercicio 1(e)

Ahora aplica las mismas transformaciones, menos las del atributo gender, al RDD de ratings.

In [26]:
#TODO: Create a DataFrame for ratings RDD
ratingsDF = <FILL IN>

display(ratingsDF)

## Parte 2: Ingenieria de caracteristicas

La ingenieria de la caracteristicas es el proceso de utilizar el conocimiento del dominio de los datos para crear las caracteristicas que hacen que los algoritmos de machine learning trabajen de forma correcta. La ingenieria de la caracteristica es fundamental en la aplicacion del machine learning, y en general es dificil y costosa.vLa ingenieria de caracteristicas es un tema informal, pero se considera esencial en el machine learning aplicado.

## Ejercicio 2(a)

Para esta PEC calcularemos unas cuantas caracteristicas muy sencillas como el numero de peliculas vistas por cada usuario, la media de sus calificaciones y su varianza.

Para obtener estas caracteristicas usaremos la funcion [groupBy()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrame.groupBy) de los DataFrames de Spark, asi como las funciones de agregacion que nos proporciona SparkQL (https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#module-pyspark.sql.functions).

Por simplicidad usaremos la funcion [alias()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.SQLContext) para poner los siguientes nombres a las columnas: NumMovies, AvgRating y VarRating.

In [29]:
#TODO: Compute aggregated values by user using groupBy and SQL functions
import pyspark.sql.functions as func

aggRatingsDF = <FILL IN>

display(aggRatingsDF)

In [30]:
# TEST compute aggregated features (2a)
counter = aggRatingsDF.count()
result = aggRatingsDF.where(aggRatingsDF.UserID == 31).first()
Test.assertEquals(counter, 6040, 'incorrect result: aggregation is incorreclty done')
Test.assertEquals(result["NumMovies"], 119, 'incorrect result: NumMovies is incorrect')
Test.assertEquals(result["AvgRating"], 3.73109243697479, 'incorrect result: AvgRating is incorrect')
Test.assertEquals(result["VarRating"], 1.0457199829084174, 'incorrect result: NumMovies is incorrect')
print result

##Ejercicio 2(b)

Ahora juntaremos las caracteristicas que hemos extraido del fichero de ratings con el fichero de usuarios. Como no queremos tener que preocuparnos por los valores nulos en las caracteristicas, solo nos quedaremos con los usuarios que hayan calificado alguna pelicula. Para realizar esto, usaremos un inner join. Encontraremos los detalles de la funcion aqui [join()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html).

In [32]:
#TODO: Compute an inner join between usersDF and aggRatingsDF
joinedDF = <FILL IN>
display(joinedDF)

In [33]:
# TEST join (2b)
counter = joinedDF.count()
Test.assertEquals(counter, 5974, 'incorrect result: join is incorreclty done')
print counter

## Ejercicio 2(c)

Ahora vamos a guardar los datos generados para poder reutilizarlos en un futuro sin tener que re-ejecutar todo el notebook, para esto ejecutaremos el siguiente codigo.

In [35]:
#TODO: Read, understand and execute the cell
sqlContext.sql("DROP TABLE IF EXISTS joinedDF")
dbutils.fs.rm("dbfs:/user/hive/warehouse/joinedDF", True)
sqlContext.registerDataFrameAsTable(joinedDF, "joinedDF")

## Parte 3: Clustering

Un algoritmo de agrupamiento (en ingles, clustering) es un procedimiento de agrupacion de una serie de vectores de acuerdo con un criterio. Esos criterios son por lo general distancia o similitud. La cercania se define en terminos de una determinada funcion de distancia, como la euclidea, aunque existen otras mas robustas o que permiten extenderla a variables discretas.

Existen dos grandes familias de clustering:

* *Agrupamiento jerarquico*, que puede ser aglomerativo o divisivo.
* *Agrupamiento no jerarquico*, en los que el numero de grupos se determina de antemano y las observaciones se van asignando a los grupos en funcion de su cercania. En esta familia existen una gran cantidad de metodos, en esta PEC usaremos el metodos de k-means (k-medias).

### k-means

K-means es un metodo de clustering, que tiene como objetivo la particion de un conjunto de _n_ observaciones en _k_ grupos en el que cada observacion pertenece al grupo cuyo valor medio es mas cercano. Una de las ventajas de este metodo es que la agrupacion del conjunto de datos puede ilustrarse en una particion del espacio de datos en [celdas de Voronoi](https://es.wikipedia.org/wiki/Pol%C3%ADgonos_de_Thiessen#Diagramas_de_Voron.C3.B3i_en_el_plano_euclidiano_.7F.27.22.60UNIQ--postMath-00000001-QINU.60.22.27.7F).

El problema es computacionalmente dificil (NP-hard). Sin embargo, hay heuristicas muy eficientes que se emplean comunmente y convergen rapidamente a un optimo local.

El algoritmo de k-means mas comun utiliza una tecnica de refinamiento iterativo, tal y como se describe a continuacion:

Dado un conjunto inicial de _k_ centroides $$ m_1^{(1)},...,m_k^{(1)} $$ el algoritmo continua alternando entre estos dos pasos:

* *Paso de asignacion:* Asigna cada observacion al grupo con la media mas cercana (es decir, la particion de las observaciones de acuerdo con el diagrama de Voronoi generado por los centroides).

$$ S_{i}^{(t)} = \\{ x_p: || x_p - m_i^{(t)} || \leq || x_p - m_j^{(t)} || \forall 1 \leq j \leq k \\} $$

* *Paso de actualizacion:* Calcular los nuevos centroides como el centroide de las observaciones en el grupo.
$$ m_i^{(t+1)} = \frac{1}{|S_i^{(t)}|} \sum^{x_j \in S_i^{(t)}} x_j$$

El algoritmo se considera que ha convergido cuando las asignaciones ya no cambian. Los centroides suelen iniciarse de forma aleatoria.

El siguiente paso es preparar los datos para aplicar el k-means. Dado que todo el dataset es numerico y consistente, esta sera una tarea sencilla y directa.

El objetivo es utilizar el metodo de clustering para determinar _k_ usuarios estandar (promedio) que representen a la totalidad de los usuarios que tenemos en nuestro dataset. El primer paso en la construccion de nuestro modelo de clustering es convertir las caracteristicas que hemos calculado en nuestro DataFrame a un vector de caracteristicas utilizando el metodo [pyspark.ml.feature.VectorAssembler()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.feature.VectorAssembler).

El VectorAssembler es una transformacion que combina una lista dada de columnas en una unico vector. Esta transformacion es muy util cuando queremos combinar caracteristicas en crudo de los datos con otras generadas al aplicar diferentes funciones sobre los datos en un unico vector de caracteristicas. Para integrar en un unico vector toda esta informacion antes de ejecutar un algoritmo de aprendizaje automatico, el VectorAssembler toma una lista con los nombres de las columnas de entrada (lista de strings) y el nombre de la columna de salida (string).

### Ejercicio 3(a)

- leer la documentacion y los ejemplos de uso de [VectorAssembler](https://spark.apache.org/docs/1.6.2/ml-features.html#vectorassembler)
- Convertir la tabla SQL `joinedDF` en un `dataset` llamado datasetDF usando la funcion table del sqlContext
- Establecer las columnas de entrada del VectorAssember: `['Age','NumMovies','AvgRating','VarRating']`
- Establecer la columnas de salida como `"features"`

In [38]:
#TODO: Replace <FILL_IN> with the appropriate code
from pyspark.ml.feature import VectorAssembler

datasetDF = <FILL IN>

vectorizer = VectorAssembler()
vectorizer.setInputCols(<FILL IN>)
vectorizer.setOutputCol(<FILL IN>)

## Ejercicio 3(b)

Guarda en cache el dataframe `datasetDF` y llamalo clusteringDF

In [40]:
#TODO: Let's cache the dataset for performance
clusteringDF = <FILL IN>

## Ejercicio 3(c)

- Lee la documentacion y los ejemplos de [k-means](https://spark.apache.org/docs/1.6.2/ml-clustering.html#k-means)
- Ejecuta la siguiente celda

In [42]:
from pyspark.ml.clustering import KMeans
from pyspark.ml import Pipeline

kmeans = KMeans()
print(kmeans.explainParams())

La siguiente celda esta basada en [Spark 2.0.1 ML Pipeline API for clustering](https://spark.apache.org/docs/2.0.1/mllib-clustering.html#k-means).

El primer paso es establecer los valores de los parametros:
- Define el numero de clusters (k) como 10
- Define el numero maximo de iteraciones a 25
- Define el random seed a 1

Ahora, crearemos el [ML Pipeline](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.Pipeline) (flujo de ejecucion) y estableceremos las fases del pipeline como vectorizar y posteriormente aplicar el algoritmo de clustering que hemos definido.

Finalmente, crearemos el modelo entrenandolo con el DataFrame `clusteringDF`.

### Ejercicio 3(d)

- Lee la documentacion [k-means](https://spark.apache.org/docs/2.0.1/mllib-clustering.html#k-means) documentation
- Completa y ejecuta la siguiente celda, asegurate de entender que es lo que sucede.

In [44]:
## TODO: Replace <FILL_IN> with the appropriate code
# Now we set the parameters for the method
kmeans = KMeans().setK(<FILL IN>).setSeed(<FILL IN>)

clusteringPipeline = Pipeline()

# We will use the new spark.ml pipeline API. If you have worked with scikit-learn this will be very familiar.
clusteringPipeline.setStages(<FILL IN>)

# Let's train on the entire dataset to see what we get
clusteringModel = clusteringPipeline.fit(<FILL IN>)

## Ejercicio 3(e)

Ejecuta la siguiente celda y observa los centroides que has obtenido y responde a la siguienes preguntas:

- Crees que los centrodides son significativos?
- Que crees que ha sucedido?

In [46]:
# Shows the result.
centers = clusteringModel.stages[1].clusterCenters()
print("Centroids: ")
for center in centers:
    print(center)

## Parte 4: Validacion de los resultados

Ahora estudiaremos como se comportan nuestras predicciones en este modelo.

Para realizar esta medicion, podemos utilizar la misma metrica de evaluacion de la PEC2, el [Error cuadratico medio](https://en.wikipedia.org/wiki/Root-mean-square_deviation) (RMSE).

Recordar que el RSME se define como: \\( RMSE = \sqrt{\frac{\sum_{i = 1}^{n} (x_i - y_i)^2}{n}}\\) donde \\(y_i\\) es el valor observado \\(x_i\\) es el valor predecido

RMSE es una medida habitual en clustering para calcular las diferencias entre los valores predecidos (centroides) y los elementos que componen ese cluster. Cuanto menor sea el RMSE, mejor sera nuestro metodo de clustering.

## Ejercicio 4(a)

El modelo de k-means de Spark proporciona un metodo para calcular la suma de las distancias al cuadrado de todos los elementos de un DataFrame a su centroide mas cercano, conocido como SSE (Sum of Square Errors). Con este valor vamos a calcular el RMSE.

In [49]:
## TODO: Replace <FILL_IN> with the appropriate code

kmeansModel = clusteringModel.stages[1]
transformedDF = clusteringModel.stages[0].transform(clusteringDF)

#Obtain the SSE
SSE = kmeansModel.computeCost(<FILL IN>)

#obtain the number of elements of the DataFrame clusteringDF
n = <FILL IN>

#compute RMSE
RMSE = <FILL IN>

In [50]:
#TEST RMSE 4(a)
Test.assertEquals(n, 5974, 'incorrect result: number of elements is wrong')
Test.assertEquals(round(RMSE,3), 29.437, 'incorrect result: RMSE error')


## Ejercicio 4(b)

Inspecciona visualmente si los centroides predecidos de algunos de los elementos de clusteringDF estan cerca de los valores reales y si tienene sentido, luego responde las siguientes preguntas:

- Como afecta el no haber normalizado los datos? 
- Hay algun atributo mas importante que los demas? Si la respuesta es afirmativa, Cual?

Ejecuta la siguiente celda para ver en que cluster se han asignado cada uno de los registros del DataFrame. Los identificadores de cluster empiezan por 0.

In [52]:
display(kmeansModel.transform(transformedDF).select(['Age','NumMovies','AvgRating','VarRating','prediction']))

## Parte 5: Normalizacion de los datos --Opcional--

Una buena practica cuando usamos metodos de machine learning basados en distancias es asegurarnos que todos los atributos estan en la misma escala. Esto no sucede en nuestro dataset ya que cada atributo tiene un rango diferente.

Prueba las siguientes opciones de normalizacion y re-ejecuta el notebook con los datos normalizados:

* **normalizacion:** Convierte todos los atributos al rango [0,1] usando la siguiente formula
$$ x' = \frac{x - min}{max-min} $$

* **standarizacion (z-scores):** Convierte todos los atributos en una distribucion normal con media = 0 y varianza = 1 usando la siguiente formula
$$ z = \frac{x - \mu}{\sigma}$$

- Tienen ahora los clusters mas sentido?
- El Error se ha reducido?